# Market Leader Analysis (Energy Only Offers/Awards)

In [1]:
# Import necessary packages
import pandas as pd
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import numpy as np
import calendar
import glob
from datetime import datetime
from dateutil.parser import parse
import os
import urllib.request
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from sklearn.linear_model import LinearRegression
#Logistic Regression & LDA
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
%matplotlib inline

#see all columns/rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Import and Clean Data
###  strip/clean column headers, datetime conversion on:

##### Environmental: Market Data, Locational Marginal Pricing
*Third Party Environmental: NOAA Hourly Weather Data*
##### Transactional: Energy Only Offers/Awards
*Third Party Transactional: Daily NASDAQ, DOWJONES, ETF prices*

In [2]:
#ERCOT
market_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_market_data.xlsx', sheet_name = 'ercot_market_data')
lmp_df = pd.read_csv('OneDrive_1_10-22-2019/ercotlmp.csv')
nodes_df = pd.read_excel('OneDrive_1_10-22-2019/ercot_nodes.xlsx')
all_offers_df = pd.concat([pd.read_csv(f) for f in glob.glob('OneDrive_1_10-22-2019/EnergyOnlyOffers/*.csv')], ignore_index = True)
all_awards_df = pd.concat([pd.read_csv(f) for f in glob.glob('OneDrive_1_10-22-2019/EnergyOnlyOfferAwards/*.csv')], ignore_index = True)

all_awards_df.columns = all_awards_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(' - ',' ')
all_offers_df.columns = all_offers_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(' - ',' ')
market_df.columns = market_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
lmp_df.columns = lmp_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nodes_df.columns = nodes_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
#Convert marketday feature from datetime type to string type
market_df['marketday'] = market_df['marketday'].dt.strftime('%m/%d/%Y')

#3rd party data
weather_df_1 = pd.read_csv('additional_data/weather_data_1.csv')
weather_df_2 = pd.read_csv('additional_data/weather_data_2.csv')
weather_df_3 = pd.read_csv('additional_data/weather_data_3.csv')
nasdaq_df = pd.read_csv('additional_data/nasdaq_data.csv')
etf_df = pd.read_csv('additional_data/etf_data.csv')
dowjones_df = pd.read_csv('additional_data/dow_jones_data.csv')

weather_df_1.columns = weather_df_1.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_2.columns = weather_df_2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
weather_df_3.columns = weather_df_3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
nasdaq_df.columns = nasdaq_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
etf_df.columns = etf_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')
dowjones_df.columns = dowjones_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('-','')

#trim
weather_df_1_trim = weather_df_1[['station','date','hourlydewpointtemperature','hourlydrybulbtemperature',
                                'hourlyprecipitation','hourlypressuretendency','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywetbulbtemperature','hourlywinddirection',
                                'hourlywindgustspeed','hourlywindspeed']]
weather_df_2_trim = weather_df_2[['station','date','hourlydewpointtemperature','hourlydrybulbtemperature',
                                'hourlyprecipitation','hourlypressuretendency','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywetbulbtemperature','hourlywinddirection',
                                'hourlywindgustspeed','hourlywindspeed']]
weather_df_3_trim = weather_df_3[['station','date','hourlydewpointtemperature','hourlydrybulbtemperature',
                                'hourlyprecipitation','hourlypressuretendency','hourlyrelativehumidity',
                                'hourlystationpressure','hourlywetbulbtemperature','hourlywinddirection',
                                'hourlywindgustspeed','hourlywindspeed']]
#rename columns
nasdaq_df = nasdaq_df.rename(columns={'date':'nasdaq_date','open':'nasdaq_open','high':'nasdaq_high',
                                     'low':'nasdaq_low','close':'nasdaq_close','adj Close':'nasdaq_adj_close',
                                     'volume':'nasdaq_volume'})
etf_df = etf_df.rename(columns={'date':'etf_date','open':'etf_open','high':'etf_high','low':'etf_low',
                                     'close':'etf_close','adj Close':'etf_adj_close','volume':'etf_volume'})
dowjones_df = dowjones_df.rename(columns={'date':'dowjones_date','open':'dowjones_open','high':'dowjones_high',
                                     'low':'dowjones_low','close':'dowjones_close','adj close':'dowjones_adj_close',
                                     'volume':'dowjones_volume'})

/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,7,8,11,14,19,25,26,27,28,29,30,31,32,33,34,35,36,41,42,43,44,49,51,52,55,56,58,69,71,72,73,75,76,88,89,95,96,97,98,99,100,101,102,103,104,105,106,107,119,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,5,7,8,11,14,19,25,26,27,28,29,30,31,34,35,36,51,55,69,71,75,76,88,89,95,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3,19,25,26,27,28,29,30,31,32,34,35,36,42,44,49,55,56,69,71,72,75,76,88,89,95,96,97,98,99,100,101,102,103,104,105,106,107

## Select Top 10 Leaders

In [3]:
#Select Leader Data from QSE Names of top 10 leaders
lead_off = all_offers_df.loc[all_offers_df['qse_name'].isin(['QREUEL','QFPLE4','QLEPPM','QRAID',
                                                                   'QSHELL','QEXELO','QTRAIL','QWAKE','QEDF26','QTTPAL'])]
lead_awd = all_awards_df.loc[all_awards_df['qse_name'].isin(['QREUEL','QFPLE4','QLEPPM','QRAID',
                                                                   'QSHELL','QEXELO','QTRAIL','QWAKE','QEDF26','QTTPAL'])]
#Select Data within timeline presented in Jeff's Power BI Dashboard (Jan 3, 2019 - July 12, 2019)
lead_awd['date'] = pd.to_datetime(lead_awd['delivery_date'])
lead_off['date'] = pd.to_datetime(lead_off['delivery_date'])
mask = (lead_awd['date'] >= '2019-01-02') & (lead_awd['date'] < '2019-07-13')
awards = lead_awd.loc[mask]
mask = (lead_off['date'] > '2019-01-02') & (lead_off['date'] <= '2019-07-12')
offers = lead_off.loc[mask]


/Users/robinsanders/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/robinsanders/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Join Tables, Create 'Data' Table for Models

In [35]:
#Join Market Condition & Location Pricing Data
market_lmp = lmp_df.merge(market_df, how = 'left', on = ['marketday','hourending','peaktype','month','year'])
market_lmp = market_lmp.drop(['datetime_y','year'],axis=1)
market_lmp = market_lmp.rename(columns={'datetime_x':'datetime'})
market_lmp_nodes = market_lmp.merge(nodes_df, how='left', left_on='settlementpoint', right_on='nodename')
awards = awards.rename(columns={'delivery_date':'marketday','hour_ending':'hourending','settlement_point':'settlementpoint'})

model_dta = awards.merge(market_lmp_nodes, how = 'left', on = ['marketday','hourending','settlementpoint'])
model_dta = model_dta.drop(columns=['iso','weatherstationid','first_dart_date','last_dart_date','equipment','voltage',
                                    'substation','nodetype','zoneid','nodename','objectid','date','datetime','offer_id'])
model_dta.loc[(model_dta.nearest_weatherstation == 'TM - Nuevo Laredo/Intl'),'nearest_weatherstation']='TX - Laredo/Intl'

In [36]:
stocks_df = nasdaq_df.merge(etf_df, how='left',left_on='nasdaq_date',right_on='etf_date')
stocks_df = stocks_df.merge(dowjones_df, how='left',left_on='nasdaq_date',right_on='dowjones_date')
stocks_df = stocks_df.drop(columns=['dowjones_date','etf_date'])
stocks_df = stocks_df.rename(columns={'nasdaq_date':'date'})
stocks_df['date'] = pd.to_datetime(stocks_df['date'],infer_datetime_format=True)
stocks_df['date'] = stocks_df['date'].dt.strftime('%m/%d/%Y')

In [37]:
weather_df = pd.concat([weather_df_1_trim, weather_df_2_trim,weather_df_3_trim])
weather_df['date'] = pd.to_datetime(weather_df['date'],infer_datetime_format=True)
weather_df['hourending'] = [d.time() for d in weather_df['date']]
mask = (weather_df['date'] >= '2019-01-02') & (weather_df['date'] < '2019-07-13')
weather_df = weather_df.loc[mask]
hours = [math.ceil((t.hour * 60 + t.minute) / 60) for t in weather_df['hourending']]
weather_df['hour'] = hours
weather_df['hour']= weather_df['hour'].apply(str).apply(int)
weather_df = weather_df.loc[(weather_df['hour'] > 0)]
weather_df['date'] = weather_df['date'].dt.strftime('%m/%d/%Y')
c_maxes = weather_df.groupby(['station', 'date','hour']).hourending.transform(max)
weather_df = weather_df.loc[weather_df.hourending == c_maxes]
weather_df['station'] = weather_df['station'].map({72267023042: 'TX - Lubbock/Intl',
                                                                 72251012924: 'TX - Corpus Christi/Intl',
                                                                 72266013962: 'TX - Abilene/Municipal', 
                                                                 72250012919: 'TX - Brownsville/Intl', 
                                                                 72351013966: 'TX - Wichita Falls/Sheppard AFB',
                                                                 72261022010: 'TX - Del Rio/Intl',
                                                                 72265023023: 'TX - Midland-Odessa',
                                                                 72253012921: 'TX - San Antonio/Intl',
                                                                 72363023047: 'TX - Amarillo/Intl',
                                                                 72248013957: 'LA - Shreveport/Regional',
                                                                 72263023034: 'TX - San Angelo/Mathis',
                                                                 72265623040: 'TX - Wink/Winkler County',
                                                                 72258013960: 'TX - Dallas/Love Field',
                                                                 72243012960: 'TX - Houston/Intercontinental',
                                                                 72261823091: 'TX - Fort Stockton',
                                                                 72252012907: 'TX - Laredo/Intl',
                                                                 74641013975: 'OK - Gage/Shattuck',
                                                                 72259303985: 'TX - Dallas-Fort Worth/Intl'})


In [38]:
weather_df = weather_df.drop(columns='hourending')
weather_df=weather_df.drop_duplicates(keep='first')

(124992, 13)

In [39]:
stocks_weather_df = weather_df.merge(stocks_df, how = 'left', on='date')
stocks_weather_df = stocks_weather_df.fillna(0.00)
stocks_weather_df=stocks_weather_df.drop_duplicates(keep='first')

In [40]:
stocks_weather_df.shape

(124992, 31)

In [41]:
model_dta.shape

(138748, 22)

In [45]:
data = model_dta.merge(stocks_weather_df.drop_duplicates(['date','hour','station']), 
                       how = 'left', 
                       left_on = ['marketday','hourending','nearest_weatherstation'], 
                       right_on=['date','hour','station'])


In [46]:
data.shape

(138748, 53)

### Create Evaluation Criterion: PnL

In [47]:
#Create PnL column for Performance Measurement/Evaluation Criterion
data['PnL'] = (data.dalmp-data.rtlmp) * data.energy_only_offer_award_in_mw
#Separate Leaders into dataframes for separate modeling
leaders_QREUEL = data.loc[data['qse_name'] == 'QREUEL']
leaders_QFPLE4 = data.loc[data['qse_name'] == 'QFPLE4']
leaders_QLEPPM = data.loc[data['qse_name'] == 'QLEPPM']
leaders_QRAID = data.loc[data['qse_name'] == 'QRAID']
leaders_QSHELL = data.loc[data['qse_name'] == 'QSHELL']
leaders_QEXELO = data.loc[data['qse_name'] == 'QEXELO']
leaders_QTRAIL = data.loc[data['qse_name'] == 'QTRAIL']
leaders_QWAKE = data.loc[data['qse_name'] == 'QWAKE']
leaders_QEDF26 = data.loc[data['qse_name'] == 'QEDF26']
leaders_QTTPAL = data.loc[data['qse_name'] == 'QTTPAL']

In [59]:
leaders_QREUEL['PnL'].sum() #Power Bi: 782,010.73
leaders_QFPLE4['PnL'].sum() #Power Bi: 544,545.72
leaders_QLEPPM['PnL'].sum() #Power Bi: 267,344.34
leaders_QRAID['PnL'].sum() #Power Bi: 193,016.14
leaders_QSHELL['PnL'].sum() #Power Bi: 179,504.23
leaders_QEXELO['PnL'].sum() #Power Bi: 155,802.45
leaders_QTRAIL['PnL'].sum() #Power Bi: 148,386.28
leaders_QWAKE['PnL'].sum() #Power Bi: 145,813.26
leaders_QEDF26['PnL'].sum() #Power Bi: 143,823.99
leaders_QTTPAL['PnL'].sum() #Power BI: 140,899.54
PnL


140899.54475

In [60]:
leaders_QREUEL_X = leaders_QREUEL[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QREUEL_y = leaders_QREUEL['PnL']  

In [64]:
leaders_QFPLE4_X = leaders_QFPLE4[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QFPLE4_y = leaders_QFPLE4['PnL']

In [66]:
leaders_QLEPPM_X = leaders_QLEPPM[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QLEPPM_y = leaders_QLEPPM['PnL']

In [68]:
leaders_QRAID_X = leaders_QRAID[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QRAID_y = leaders_QRAID['PnL']

In [70]:
leaders_QSHELL_X = leaders_QSHELL[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QSHELL_y = leaders_QSHELL['PnL']

In [72]:
leaders_QEXELO_X = leaders_QEXELO[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QEXELO_y = leaders_QEXELO['PnL']

In [74]:
leaders_QTRAIL_X = leaders_QTRAIL[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QTRAIL_y = leaders_QTRAIL['PnL']

In [76]:
leaders_QWAKE_X = leaders_QWAKE[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QWAKE_y = leaders_QWAKE['PnL']

In [78]:
leaders_QEDF26_X = leaders_QEDF26[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QEDF26_y = leaders_QEDF26['PnL']

In [80]:
leaders_QTTPAL_X = leaders_QTTPAL[['hour',
                                'energy_only_offer_award_in_mw','settlement_point_price',
                                'rtlmp','dalmp',
                                'gr_panhandle_wind_stwpf', 'panhandle_ercot_rt_generic_constraints',
                                'ercot_total_resource_cap_out','ercot_gen_resource','ercot_rtload',
                                'ercot_original_load_forecast','ercot_wind_rti','ercot_wind_stwpf_orig',
                                'ercot_total_resource_cap_out.1','ercot_renew_resource_cap_out',
                                'ercot_renew_resource_cap_out',
                                 #'hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation',
                                'hourlypressuretendency','hourlyrelativehumidity',#'hourlystationpressure',
                                'hourlywetbulbtemperature',#'hourlywinddirection','hourlywindgustspeed',
                                #'hourlywindspeed',
                                'nasdaq_open','nasdaq_high','nasdaq_low','nasdaq_close','adj_close_x',
                                'nasdaq_volume','etf_open','etf_high','etf_low','etf_close','adj_close_y',
                                'etf_volume','dowjones_open','dowjones_high','dowjones_low','dowjones_close','adj_close','dowjones_volume',]]
leaders_QTTPAL_y = leaders_QTTPAL['PnL']

In [61]:
leaders_QREUEL.fillna(0)
leaders_QREUEL = leaders_QREUEL.astype('float64', errors='ignore')

In [ ]:
leaders_QREUEL.fillna(0)
leaders_QREUEL['hourlydewpointtemperature'] = leaders_QREUEL['hourlydewpointtemperature'].str.extract('(\d+)', expand=False)
#leaders_QREUEL[['hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation','hourlystationpressure','hourlywinddirection','hourlywindgustspeed','hourlywindspeed']] = leaders_QREUEL[['hourlydewpointtemperature','hourlydrybulbtemperature','hourlyprecipitation','hourlystationpressure','hourlywinddirection','hourlywindgustspeed','hourlywindspeed']].str.extract('(\d+)', expand=False)
#leaders_QREUEL.hourlydewpointtemperature #.astype('int32',errors='raise')
leaders_QREUEL.dtypes

# Model Test 1: Linear Regression

In [63]:
leaders_QREUEL_X = leaders_QREUEL_X.fillna(0)
reg = LinearRegression().fit(leaders_QREUEL_X,leaders_QREUEL_y)
reg.coef_
reg.score(leaders_QREUEL_X, leaders_QREUEL_y)

0.42060257756421326

In [65]:
leaders_QFPLE4_X = leaders_QFPLE4_X.fillna(0)
reg = LinearRegression().fit(leaders_QFPLE4_X,leaders_QFPLE4_y)
reg.coef_
reg.score(leaders_QFPLE4_X, leaders_QFPLE4_y)

0.3229764214956178

In [67]:
leaders_QLEPPM_X = leaders_QLEPPM_X.fillna(0)
reg = LinearRegression().fit(leaders_QLEPPM_X,leaders_QLEPPM_y)
reg.coef_
reg.score(leaders_QLEPPM_X, leaders_QLEPPM_y)

0.6788657247893316

In [69]:
leaders_QRAID_X = leaders_QRAID_X.fillna(0)
reg = LinearRegression().fit(leaders_QRAID_X,leaders_QRAID_y)
reg.coef_
reg.score(leaders_QRAID_X, leaders_QRAID_y)

0.1832498556127229

In [71]:
leaders_QSHELL_X = leaders_QSHELL_X.fillna(0)
reg = LinearRegression().fit(leaders_QSHELL_X,leaders_QSHELL_y)
reg.coef_
reg.score(leaders_QSHELL_X, leaders_QSHELL_y)

0.16637722234598218

In [73]:
leaders_QEXELO_X = leaders_QEXELO_X.fillna(0)
reg = LinearRegression().fit(leaders_QEXELO_X,leaders_QEXELO_y)
reg.coef_
reg.score(leaders_QEXELO_X, leaders_QEXELO_y)

0.08153647038030054

In [75]:
leaders_QTRAIL_X = leaders_QTRAIL_X.fillna(0)
reg = LinearRegression().fit(leaders_QTRAIL_X,leaders_QTRAIL_y)
reg.coef_
reg.score(leaders_QTRAIL_X, leaders_QTRAIL_y)

0.9997463420361379

In [77]:
leaders_QWAKE_X = leaders_QWAKE_X.fillna(0)
reg = LinearRegression().fit(leaders_QWAKE_X,leaders_QWAKE_y)
reg.coef_
reg.score(leaders_QWAKE_X, leaders_QWAKE_y)

0.7965887690634498

In [79]:
leaders_QEDF26_X = leaders_QEDF26_X.fillna(0)
reg = LinearRegression().fit(leaders_QEDF26_X,leaders_QEDF26_y)
reg.coef_
reg.score(leaders_QEDF26_X, leaders_QEDF26_y)

0.4265418406526341

In [81]:
leaders_QTTPAL_X = leaders_QTTPAL_X.fillna(0)
reg = LinearRegression().fit(leaders_QTTPAL_X,leaders_QTTPAL_y)
reg.coef_
reg.score(leaders_QTTPAL_X, leaders_QTTPAL_y)

0.8834939863759151